In [83]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 1000)
sns.set_style("whitegrid")

II - PRE-TRAITEMENT

Lecture des csv

In [84]:
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")
store = pd.read_csv("../data/store.csv")

C:\Users\mandr\AppData\Local\Temp\ipykernel_36828\801007824.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv("../data/train.csv")


Supprimer les jours où le magasin est fermé

In [85]:
train = train[train["Open"] == 1].copy()

Remplacer les NA numériques par la médiane

In [86]:
for col in train.select_dtypes(include=["int64", "float64"]).columns:
    train[col] = train[col].fillna(train[col].median())

Remplacer les NA valeurs catégorielles par "Unknown"

In [87]:
for col in train.select_dtypes(include=["object"]).columns:
    train[col] = train[col].fillna("Unknown")

Fusionner les informations des magasins (store.csv) avec les ventes

In [88]:
train = train.merge(store, on="Store", how="left")
train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,5,2015-07-31,6064,625,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,5,2015-07-31,8314,821,1,1,0,1,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,5,2015-07-31,4822,559,1,1,0,1,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


Transformer la colonne Date en :
Année
Mois
Jour de la semaine


In [89]:
train["Date"] = pd.to_datetime(train["Date"])
train["Year"] = train["Date"].dt.year
train["Month"] = train["Date"].dt.month
train["DayOfWeek"] = train["Date"].dt.dayofweek
train[["Year", "Month", "DayOfWeek"]].head()

,Year,Month,DayOfWeek
0,2015,7,4
1,2015,7,4
2,2015,7,4
3,2015,7,4
4,2015,7,4


Trier par magasin puis par date et réinitialiser les indexes

In [90]:
train = train.sort_values(["Store", "Date"])
train = train.reset_index(drop=True)
train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Year,Month
0,1,2,2013-01-02,5530,668,1,0,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,2013,1
1,1,3,2013-01-03,4327,578,1,0,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,2013,1
2,1,4,2013-01-04,4486,619,1,0,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,2013,1
3,1,5,2013-01-05,4997,635,1,0,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,2013,1
4,1,0,2013-01-07,7176,785,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,2013,1


encoding des colonnes texte (pour sklearn)

In [91]:
train = pd.get_dummies(train, drop_first=True)

séparation train / validation basée sur le temps

In [92]:
# Choisir une date de séparation pour le train/test
date_split = "2015-06-01"

# Séparer les données en train et test
train_set = train[train["Date"] < date_split].copy()
val_set = train[train["Date"] >= date_split].copy()

# Vérifier
print("Train :", train_set["Date"].min(), "→", train_set["Date"].max())
print("Validation :", val_set["Date"].min(), "→", val_set["Date"].max())

Train : 2013-01-01 00:00:00 → 2015-05-31 00:00:00
Validation : 2015-06-01 00:00:00 → 2015-07-31 00:00:00


III - PLAN D'EXPERIENCE

Séparer X et y puis entraîner le modèle

In [93]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(random_state=42)
# voir quelles colonnes sont des dates
datetime_cols = X_train.select_dtypes(include=["datetime64[ns]", "datetime64[ns, UTC]"]).columns
print("Colonnes datetime :", list(datetime_cols))

# les supprimer de X_train et X_val
X_train = X_train.drop(columns=datetime_cols)
X_val = X_val.drop(columns=datetime_cols)
model.fit(X_train, y_train)

Colonnes datetime : []


ValueError: could not convert string to float: 'a'